In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
page = requests.get("https://stardewvalleywiki.com/Crops")
page

<Response [200]>

In [4]:
bs = BeautifulSoup(page.text, 'html.parser')
bs


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Crops - Stardew Valley Wiki</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"333b195b20cc95b3cea5ac88","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Crops","wgTitle":"Crops","wgCurRevisionId":178287,"wgRevisionId":178287,"wgArticleId":345,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Crops","Achievements"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"Crops","wgRelevantArticleId":345,"wgIsProbablyEditable":!1,"wgRelevantPageIsProbablyEditable":!1,

In [5]:
crops = bs.find_all('span', {'class':'mw-headline'})

In [6]:
crop_list = []
words_to_remove = ['Fiber','seeds','Seeds', 'Basics', 'Monsters', 'Crows','Crops', 'Farm Animals', 'and', 'Trellis', 'Grow ', 'End ', 'Giant ', 'Quality', 'Gold ', 'Artisan ', 'Achievements', 'Notes', 'External ', 'References', 'History', 'Basics']
for crop in crops:
    crop_text = crop.get_text()
    if not any(word in crop_text for word in words_to_remove):
        crop_list.append(crop_text)

In [7]:
bs.find_all('tr')

[<tr>
 <th colspan="5">Giant Crops
 </th></tr>,
 <tr>
 <th>Cauliflower
 </th>
 <th>Melon
 </th>
 <th>Pumpkin
 </th>
 <th>Powdermelon
 </th>
 <th>Qi Fruit
 </th></tr>,
 <tr>
 <td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Cauliflower.png"><img alt="Giant Cauliflower.png" decoding="async" height="116" src="/mediawiki/images/6/6c/Giant_Cauliflower.png" width="96"/></a></div></div>
 </td>
 <td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Melon.png"><img alt="Giant Melon.png" decoding="async" height="116" src="/mediawiki/images/b/bc/Giant_Melon.png" width="96"/></a></div></div>
 </td>
 <td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Pumpkin.png"><img alt="Giant Pumpkin.png" decoding="async" height="116" src="/mediawiki/images/c/c8/Giant_Pumpkin.png" width="96"/></a></div></div>
 </td>
 <td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Powdermelon.png"><img al

In [8]:
bs.find_all('td')

[<td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Cauliflower.png"><img alt="Giant Cauliflower.png" decoding="async" height="116" src="/mediawiki/images/6/6c/Giant_Cauliflower.png" width="96"/></a></div></div>
 </td>,
 <td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Melon.png"><img alt="Giant Melon.png" decoding="async" height="116" src="/mediawiki/images/b/bc/Giant_Melon.png" width="96"/></a></div></div>
 </td>,
 <td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Pumpkin.png"><img alt="Giant Pumpkin.png" decoding="async" height="116" src="/mediawiki/images/c/c8/Giant_Pumpkin.png" width="96"/></a></div></div>
 </td>,
 <td><div class="center"><div class="floatnone"><a class="image" href="/File:Giant_Powdermelon.png"><img alt="Giant Powdermelon.png" decoding="async" height="122" src="/mediawiki/images/2/20/Giant_Powdermelon.png" width="96"/></a></div></div>
 </td>,
 <td><div class="center"><

In [9]:
price_tables = bs.find_all('table', {'class': 'no-wrap'})

In [10]:
price = []

for i in price_tables:
    rows = i.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        for cell in cells:
            if 'g' in cell.text:
                price.append(cell.text.strip())

In [11]:
def split_into_chunks(data, chunk_size=4):
    return [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

sell_chunks = split_into_chunks(price, 4)

In [12]:
rows = bs.find_all('td')

In [13]:
rows_with_days = []
for row in rows:
    if 'Total' in row.get_text():
        rows_with_days.append(row.get_text(strip=True))


In [16]:
df_crops = pd.DataFrame(crop_list, columns=['Crops'])
df_sell_chunks = pd.DataFrame(sell_chunks)
df_sell_chunks.columns = ['Low Qual', 'Average Qual', 'Good Qual', 'Best Qual']
df_rows_with_days = pd.DataFrame(rows_with_days, columns=['Growth Time'])

In [17]:
combined_df = pd.concat([df_crops, df_sell_chunks, df_rows_with_days], axis=1)
combined_df

,Crops,Low Qual,Average Qual,Good Qual,Best Qual,Growth Time
0,Blue Jazz,50g,62g,75g,100g,Total: 7 days
1,Carrot,35g,43g,52g,70g,Total: 3 days
2,Cauliflower,175g,218g,262g,350g,Total: 12 days
3,Coffee Bean,15g,18g,22g,30g,Total: 10 days
4,Garlic,60g,75g,90g,120g,Total: 4 days
5,Green Bean,40g,50g,60g,80g,Total: 10 days
6,Kale,110g,137g,165g,220g,Total: 6 days
7,Parsnip,35g,43g,52g,70g,Total: 4 days
8,Potato,80g,100g,120g,160g,Total: 6 days
9,Rhubarb,220g,275g,330g,440g,Total: 13 days
